In [26]:
import pandas as pd
import numpy as np

# 데이터 불러오기
df = pd.read_csv('/Users/t2023-m0069/Desktop/Github/Omakase/Courses.csv')

In [27]:
# grade 공백 → NaN, 숫자 변환, 1.01 → 1.0
df['grade'] = pd.to_numeric(df['grade'].replace(' ', np.nan), errors='coerce')
df['grade'] = df['grade'].replace(1.01, 1.0)
# NaN 유지하면서 정수형 변환 (결측치 있을 수 있으므로 object로 남을 수 있음)
df['grade'] = df['grade'].apply(lambda x: int(x) if pd.notnull(x) else np.nan)

# viewed가 False인데 활동지표가 모두 채워진 경우 제거
conditions = ['nevents', 'ndays_act', 'nchapters', 'nforum_posts']
cond_all_filled = (df['viewed'] == False) & df[conditions].notnull().all(axis=1)
df = df[~cond_all_filled]

# viewed가 False인데 일부 지표만 채워진 경우 → 해당 값들 0으로
cond_partial_filled = (df['viewed'] == False) & df[conditions].notnull().any(axis=1)
df.loc[cond_partial_filled, conditions] = df.loc[cond_partial_filled, conditions].fillna(0)

# last_event_DI가 start_time_DI보다 앞선 경우 삭제
if all(col in df.columns for col in ['start_time_DI', 'last_event_DI']):
    date_error = df['last_event_DI'] < df['start_time_DI']
    date_error = date_error & df['last_event_DI'].notna() & df['start_time_DI'].notna()

# YoB 이상치 제거 (IQR 기준 상위만)
q3 = df['YoB'].quantile(0.75)
iqr = q3 - df['YoB'].quantile(0.25)
upper_bound = q3 + 1.5 * iqr
df = df[df['YoB'] <= upper_bound]

# 나이 파생 변수 생성
df['age'] = 2025 - df['YoB']
df['age'] = df['age'].where(df['YoB'].notnull(), 'Unknown')

# 결측치 Unknown 처리
df['LoE_DI'] = df['LoE_DI'].fillna('Unknown')
df['gender'] = df['gender'].fillna('Unknown')

# roles 컬럼 제거
df = df.drop(columns=['roles'])

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 542613 entries, 19330 to 641137
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              542613 non-null  int64  
 1   course_id          542613 non-null  object 
 2   userid_DI          542613 non-null  object 
 3   registered         542613 non-null  int64  
 4   viewed             542613 non-null  int64  
 5   explored           542613 non-null  int64  
 6   certified          542613 non-null  int64  
 7   final_cc_cname_DI  542613 non-null  object 
 8   LoE_DI             542613 non-null  object 
 9   YoB                542613 non-null  float64
 10  gender             542613 non-null  object 
 11  grade              493392 non-null  float64
 12  start_time_DI      542613 non-null  object 
 13  last_event_DI      384982 non-null  object 
 14  nevents            475869 non-null  float64
 15  ndays_act          485099 non-null  float64
 16  npl